# Straight forward solution to have a running classification model that we can later benchmark against

## Data Prep

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

d:\Visual Studio Projects\AI-Content-Moderator\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_path = "../data/jigsaw/train.csv"
test_path = "../data/jigsaw/test.csv"
test_labels_path = "../data/jigsaw/test_labels.csv"

In [6]:
train = pd.read_csv(train_path)
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df_train = train.sample(frac=0.1, random_state=42)  

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def encode_text(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')

tokenized_train = encode_text(df_train['comment_text'])
y_train = torch.LongTensor(df_train[label_cols].values)

train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

## Setup model


In [5]:
model_name = 'bert-base-uncased'
learning_rate = 2e-5

In [17]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_cols))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model.train()
for epoch in range(2):  # 2 epochs
    for batch in train_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        print(f"Input IDs shape: {input_ids.shape}, Attention mask shape:{attention_mask.shape}, Labels shape: {labels.shape}")
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(outputs.logits, labels.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask shape:torch.Size([16, 128]), Labels shape: torch.Size([16, 6])
Input IDs shape: torch.Size([16, 128]), Attention mask s

In [ ]:
model.save_pretrained('../models/bert_finetuned_baseline')

In [2]:
# Load saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = '../models/bert_finetuned_baseline'
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Evaluate model

In [7]:
df_test = pd.read_csv(test_path)
df_labels = pd.read_csv(test_labels_path)
df_test = df_test.merge(df_labels, on='id')

# filter out the not-included comments for evaluation
df_test = df_test[df_test["toxic"]!=-1]
print(f'New test dataset: {df_test.shape}')

y_test = torch.LongTensor(df_test[label_cols].values)
tokenized_test = encode_text(df_test['comment_text'])

test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], y_test)
test_loader = DataLoader(test_dataset, batch_size=16)

New test dataset: (63978, 8)


In [8]:
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.sigmoid(outputs.logits) > 0.5
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [9]:
for i, label in enumerate(label_cols):
    f1 = f1_score([t[i] for t in true_labels], [p[i] for p in predictions], average='micro')  # Per column
    print(f"F1-Score for {label}: {f1}")

F1-Score for toxic: 0.9248179061552408
F1-Score for severe_toxic: 0.9936228078401951
F1-Score for obscene: 0.9588452280471412
F1-Score for threat: 0.9967019913095126
F1-Score for insult: 0.959892463034168
F1-Score for identity_hate: 0.9891525211791553


* F1-Score for toxic: 0.9248179061552408
* F1-Score for severe_toxic: 0.9936228078401951
* F1-Score for obscene: 0.9588452280471412
* F1-Score for threat: 0.9967019913095126
* F1-Score for insult: 0.959892463034168
* F1-Score for identity_hate: 0.9891525211791553

In [10]:

for i, label in enumerate(label_cols):
    acc = accuracy_score([t[i] for t in true_labels], [p[i] for p in predictions])
    prec = precision_score([t[i] for t in true_labels], [p[i] for p in predictions], average='micro')
    recall = recall_score([t[i] for t in true_labels], [p[i] for p in predictions], average='micro')
    print(f"Accuracy for {label}: {acc}")
    print(f"Precision for {label}: {prec}")
    print(f"Recall for {label}: {recall}")

Accuracy for toxic: 0.9248179061552408
Precision for toxic: 0.9248179061552408
Recall for toxic: 0.9248179061552408
Accuracy for severe_toxic: 0.9936228078401951
Precision for severe_toxic: 0.9936228078401951
Recall for severe_toxic: 0.9936228078401951
Accuracy for obscene: 0.9588452280471412
Precision for obscene: 0.9588452280471412
Recall for obscene: 0.9588452280471412
Accuracy for threat: 0.9967019913095126
Precision for threat: 0.9967019913095126
Recall for threat: 0.9967019913095126
Accuracy for insult: 0.959892463034168
Precision for insult: 0.959892463034168
Recall for insult: 0.959892463034168
Accuracy for identity_hate: 0.9891525211791553
Precision for identity_hate: 0.9891525211791553
Recall for identity_hate: 0.9891525211791553


It seems weird that all metrics show the same numbers - will save the results and calculate the metrics differentely

In [12]:
for i in range(100):
    print(f'Prediction: {predictions[i]} vs True labels: {true_labels[i]}')

Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [ True False False False False False] vs True labels: [1 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False False False] vs True labels: [0 0 0 0 0 0]
Prediction: [False False False False Fal

In [14]:
import numpy as np
val_true = np.vstack(true_labels)
val_preds = np.vstack(predictions)

import pandas as pd
df_true = pd.DataFrame(true_labels, columns=label_cols)
df_preds = pd.DataFrame(predictions, columns=label_cols)
df_true.to_csv('../outputs/true_labels_with_cols.csv', index=False)
df_preds.to_csv('../outputs/predictions_with_cols.csv', index=False)